# Partie 1

In [386]:
import pandas as pd
import re

def extract_data(input_path, pattern):
    with open(input_path, 'r') as file:
        content = file.read()
    matches = re.findall(pattern, content, re.MULTILINE)
    data = []
    for match in matches:
        numbers = match[1].replace('\n', ' ').split()
        numbers_float = [float(number) for number in numbers]
        data.append(numbers_float)
    return pd.DataFrame(data)

rocks_path = 'data/sonar.rocks'
mines_path = 'data/sonar.mines'


train_pattern = r'\*(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'
test_pattern = r'^(?!\*)(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'


rocks_train_df = extract_data(rocks_path, train_pattern)
mines_train_df = extract_data(mines_path, train_pattern)
rocks_train_df['Label'] = 'R'  
mines_train_df['Label'] = 'M' 
train_df = pd.concat([rocks_train_df, mines_train_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

rocks_test_df = extract_data(rocks_path, test_pattern)
mines_test_df = extract_data(mines_path, test_pattern)
rocks_test_df['Label'] = 'R'  
mines_test_df['Label'] = 'M' 
test_df = pd.concat([rocks_test_df, mines_test_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

## Algorithme d'entraînement

In [387]:
import numpy as np

def minimerror_perceptron(training_features, training_labels, epochs, learning_rate, T_initial, T_decrease_factor):
    
    weights = np.zeros(training_features.shape[1] + 1)
    T = T_initial
    
    for _ in range(epochs):
        
        update = np.zeros(training_features.shape[1] + 1)
        
        for inputs, label in zip(training_features, training_labels):
            
            activation = np.dot(inputs, weights[1:]) + weights[0]
            predicted_label = np.tanh(activation / T)
            
            # Mise à jour de la règle Minimerror, en utilisant la tangente hyperbolique pour la "température"
            error = label - predicted_label
            update[1:] += learning_rate * error * inputs
            update[0] += learning_rate * error
        
        weights += update
        
        # Diminution de la "température" après chaque époque pour affiner l'apprentissage
        T *= T_decrease_factor
    
    return weights

train_df['Label'] = train_df['Label'].apply(lambda x: 1 if x == 'M' else -1)
test_df['Label'] = test_df['Label'].apply(lambda x: 1 if x == 'M' else -1)


train_features = train_df.iloc[:, :-1].values
train_labels = train_df.iloc[:, -1].values
weights = minimerror_perceptron(train_features, train_labels, 600, 0.001, T_initial=80, T_decrease_factor=0.99)
print (weights)


[-0.7258497   0.21159445  0.38491744  0.32606422  0.53436995  0.56897422
  0.21526413  0.05270522 -0.25925507  0.44453582  0.5752605   0.82109167
  0.86641213  0.82248482  0.28308209 -0.28716421 -0.91566584 -0.54622001
 -0.21480961  0.23341056  0.20456587  0.3123386   0.38649658  0.09747376
  0.19790741 -0.14322522 -0.36219101 -0.1941907   0.04100954  0.25692631
  0.18740009 -0.76457626  0.05496442 -0.05310656 -0.41353865 -0.67201051
 -0.86356772 -0.62081879  0.27156894  0.21043719 -0.51119612  0.111341
  0.49329699  0.55662578  0.89377351  1.40883839  1.18875956  0.84899708
  0.75775999  0.56093144  0.04599946  0.09881678  0.09159663  0.02209409
  0.05353741 -0.00682674  0.00601123 -0.0147758   0.04354615  0.03551326
  0.0123419 ]


Ea et Eg

In [388]:
def predict_perceptron(inputs, weights):
    activation = np.dot(inputs, weights[1:]) + weights[0]
    return 1 if activation >= 0 else -1


def calculate_accuracy(features, labels, weights):
    predictions = [predict_perceptron(x, weights) for x in features]
    correct_predictions = sum(pred == label for pred, label in zip(predictions, labels))
    accuracy = correct_predictions / len(labels)
    return accuracy


test_features = test_df.iloc[:, :-1].values
test_labels = test_df.iloc[:, -1].values


training_accuracy = calculate_accuracy(train_features, train_labels, weights)
testing_accuracy = calculate_accuracy(test_features, test_labels, weights)


Ea = 1 - training_accuracy
Eg = 1 - testing_accuracy

print(f"Erreur d'apprentissage (Ea) : {Ea}")
print(f"Erreur de généralisation (Eg) : {Eg}")


Erreur d'apprentissage (Ea) : 0.1923076923076923
Erreur de généralisation (Eg) : 0.14423076923076927


2 attempt

In [389]:
import numpy as np

def minimerror_perceptron2(training_features, training_labels, epochs, learning_rate, T_initial, T_decrease_factor):
    # Initialisation des poids avec une petite valeur non nulle pour éviter la division par zéro.
    weights = np.random.uniform(low=-0.5, high=0.5, size=training_features.shape[1] + 1)
    T = T_initial  # Température initiale

    for epoch in range(epochs):
        for inputs, label in zip(training_features, training_labels):
            # Calcul de la stabilité pour l'exemple actuel
            gamma = label * (np.dot(inputs, weights[1:]) + weights[0])
            
            # Calcul de la contribution à la fonction de coût pour cet exemple
            V = 0.5 * (1 - np.tanh(gamma / (2 * T)))
            
            # Calcul de la dérivée de V par rapport à gamma
            dV_dgamma = -0.5 * (1 - np.tanh(gamma / (2 * T))**2) * (1 / (2 * T))
            
            # Mise à jour des poids - la descente de gradient sur la fonction de coût
            weights[1:] -= learning_rate * dV_dgamma * label * inputs
            weights[0] -= learning_rate * dV_dgamma * label
        
        # Diminution de la température après chaque époque pour affiner l'apprentissage
        T *= T_decrease_factor
    
    return weights

# Paramètres de l'algorithme (à ajuster selon le problème)
epochs = 1000
learning_rate = 0.001
T_initial = 100.0  # Température initiale élevée
T_decrease_factor = 0.99  # Facteur de diminution de la température

weights2 = minimerror_perceptron2(train_features, train_labels, epochs, learning_rate, T_initial, T_decrease_factor)


Ea et Eg 2

In [390]:

training_accuracy2 = calculate_accuracy(train_features, train_labels, weights2)
testing_accuracy2 = calculate_accuracy(test_features, test_labels, weights2)


Ea2 = 1 - training_accuracy2
Eg2 = 1 - testing_accuracy2

print(f"Erreur d'apprentissage (Ea) : {Ea2}")
print(f"Erreur de généralisation (Eg) : {Eg2}")

Erreur d'apprentissage (Ea) : 0.17307692307692313
Erreur de généralisation (Eg) : 0.22115384615384615


3

In [391]:
import numpy as np

def calculate_error(weights, training_features, training_labels):
    errors = 0
    for inputs, label in zip(training_features, training_labels):
        inputs_with_bias = np.insert(inputs, 0, 1)
        prediction = np.sign(np.dot(weights, inputs_with_bias))
        if prediction != label:
            errors += 1
    return errors

def minimerror_perceptron4(training_features, training_labels, epochs, initial_learning_rate, T_plus_initial, T_minus_initial, T_decrease_factor, patience, decay):
    weights = np.random.uniform(low=0.3, high=0.4, size=(training_features.shape[1] + 1))
    T_plus = T_plus_initial
    T_minus = T_minus_initial
    previous_errors = float('inf')
    no_improvement_count = 0
    learning_rate = initial_learning_rate

    for epoch in range(epochs):
        for inputs, label in zip(training_features, training_labels):
            inputs_with_bias = np.insert(inputs, 0, 1)
            stability = (np.dot(inputs_with_bias, weights) * label) / np.linalg.norm(weights[1:])
            T = T_plus if stability >= 0 else T_minus
            derivative = -inputs_with_bias * label / (4 * T * np.cosh(stability / (2 * T))**2)
            weights[1:] -= learning_rate * derivative[1:] / np.linalg.norm(weights[1:])
            weights[0] -= learning_rate * derivative[0]

        current_errors = calculate_error(weights, training_features, training_labels)

        if current_errors < previous_errors:
            previous_errors = current_errors
            no_improvement_count = 0
            # Diminuez les températures lorsque les erreurs s'améliorent
            T_plus *= T_decrease_factor
            T_minus *= T_decrease_factor
        else:
            no_improvement_count += 1
            # Augmentez les températures si les erreurs augmentent ou stagnent
            T_plus /= T_decrease_factor
            T_minus /= T_decrease_factor

        if no_improvement_count >= patience:
            print(f'Arrêt prématuré après {epoch+1} époques en raison de l\'absence d\'amélioration.')
            break

        # Réduisez le taux d'apprentissage progressivement
        learning_rate *= (1.0 / (1.0 + decay * epoch))

        print(f'Epoch {epoch+1}/{epochs}, Erreurs: {current_errors}, T+: {T_plus}, T-: {T_minus}, Learning Rate: {learning_rate}')

    return weights

# Exemple d'utilisation de la méthode minimerror_perceptron :
# Vous devrez définir training_features, training_labels et ajuster les hyperparamètres ci-dessous selon votre cas d'usage.

# Exemple d'utilisation de la méthode minimerror_perceptron:
weights4 = minimerror_perceptron4(train_features, train_labels, epochs=10000, initial_learning_rate=0.02, T_plus_initial=100, T_minus_initial=1, T_decrease_factor=0.99, patience=200, decay=1e-6)



training_accuracy3 = calculate_accuracy(train_features, train_labels, weights4)
testing_accuracy3 = calculate_accuracy(test_features, test_labels, weights4)


Ea3 = 1 - training_accuracy3
Eg3 = 1 - testing_accuracy3

print(f"Erreur d'apprentissage (Ea) : {Ea3}")
print(f"Erreur de généralisation (Eg) : {Eg3}")

Epoch 1/10000, Erreurs: 55, T+: 99.0, T-: 0.99, Learning Rate: 0.02
Epoch 2/10000, Erreurs: 55, T+: 100.0, T-: 1.0, Learning Rate: 0.019999980000020002
Epoch 3/10000, Erreurs: 55, T+: 101.01010101010101, T-: 1.0101010101010102, Learning Rate: 0.01999994000014
Epoch 4/10000, Erreurs: 55, T+: 102.03040506070809, T-: 1.020304050607081, Learning Rate: 0.0199998800005
Epoch 5/10000, Erreurs: 55, T+: 103.06101521283645, T-: 1.0306101521283648, Learning Rate: 0.0199998000013
Epoch 6/10000, Erreurs: 55, T+: 104.10203556852167, T-: 1.041020355685217, Learning Rate: 0.019999700002799985
Epoch 7/10000, Erreurs: 55, T+: 105.15357128133502, T-: 1.0515357128133505, Learning Rate: 0.019999580005319952
Epoch 8/10000, Erreurs: 55, T+: 106.21572856700507, T-: 1.062157285670051, Learning Rate: 0.019999440009239887
Epoch 9/10000, Erreurs: 44, T+: 105.15357128133502, T-: 1.0515357128133505, Learning Rate: 0.019999280014999767
Epoch 10/10000, Erreurs: 41, T+: 104.10203556852167, T-: 1.041020355685217, Learn